In [4]:
def state_update(S):
    S_new = [S[i+1] for i in range(len(S)-1)] + [0]
       
    S_new[255] = S[0] + S[41]*S[70] + s0 + s0
    S_new[251] = S[252] + S[42]*S[83] + S[8] + s0 + s0
    S_new[247] = S[248] + S[44]*S[102] + S[40] + s0 + s0
    S_new[243] = S[244] + S[43]*S[118] + S[103] + s0 + s0
    S_new[239] = S[240] + S[46]*S[141] + S[117] + s0 + s0
    S_new[235] = S[236] + S[67]*S[90]*S[110]*S[137] + s0 + s0
    S_new[231] = S[232] + S[50]*S[159] + S[189] + s0 + s0
    S_new[217] = S[218] + S[3]*S[32] + s0 + s0
    S_new[213] = S[214] + S[4]*S[45] + s0 + s0
    S_new[209] = S[210] + S[6]*S[64] + s0 + s0
    S_new[205] = S[206] + S[5]*S[80] + s0 + s0
    S_new[201] = S[202] + S[8]*S[103] + s0 + s0
    S_new[197] = S[198] + S[29]*S[52]*S[72]*S[99] + s0 + s0
    S_new[193] = S[194] + S[12]*S[121] + s0 + s0
    
    return S_new

def keystream(S):
    z = S[80] + S[99] + S[137] + S[227] + S[222] + S[187] + S[243]*S[217] + S[247]*S[231] \
    + S[213]*S[235] + S[255]*S[251] + S[181]*S[239] + S[174]*S[44] + S[164]*S[29] \
    + S[255]*S[247]*S[243]*S[213]*S[181]*S[174] + s0 + s0
    
    return z

def Encryption():
    K = [randint(0,1) for i in range(128)]
    IV = [randint(0,1) for i in range(96)]
    S = K[:] + IV[:] + [1 for i in range(224,255,1)] + [0]
    
    for loop in range(256):
        z = keystream(S)
        S = state_update(S)
        S[255] = (S[255] + z)
        S[217] = (S[217] + z)
        
    keys = 100
    Z = [0]*keys
    for loop in range(keys):
        Z[loop] = keystream(S)
        S = state_update(S)
        
    return S,Z
        
    
def intersection(A,B):
    temp = []
    for i in range(len(A)):
        if A[i] in B:
            temp.append(A[i])
    return temp


In [8]:
filename = 'Espresso_TMDTO_Output.txt'

keys = 15
import time
start2 = time.time()

state_len = 256
B = BooleanPolynomialRing(state_len,['s%d'%i for i in range(state_len)])
B.inject_variables()
g = list(B.gens())


S = g[:]

R = []
G = []
F = []

for i in F:
    S[i] = 0




linear = []
linear_all = []
n1 = 0
Z = [0]*keys

for loop in range(keys):
    Z[loop] = keystream(S)
    monom = list(Z[loop].monomials())
    lin = []
    for i in monom:
        if i.degree() > 1:
            n1 += 1
        if i.degree() == 1 and g.index(i) not in linear_all:
            lin = lin + [g.index(i)]
    linear.append(lin)
    linear_all = linear_all + lin
    print("loop 1st part ",loop)
    if loop != keys-1:
        S = state_update(S)
print("1st part done")

#Assigning variables to the registers
S = g[:]

for i in F:
    S[i] = 0



import gurobipy as gp
from gurobipy import GRB
import numpy as np
model = gp.Model()


v = [0,1,2]             # Temporary variable to be used for gurobi variable definition
aux = [i for i in range(n1)]

#Let D, F, G represents the class of determined/recovery, Fixed bit, and Guess bit
#Since these are set and we need variables to denote R,F and G therefore let 2=R, 3=F and 4=G
#x represents variables for each register and m contains 3 variables for each x[i] i.e. for R,F and G
x = model.addVars(len(S), lb = 2, ub = 4, vtype = GRB.INTEGER, name = "x")
m = model.addVars(len(S), len(v), vtype = GRB.BINARY, name = "m")
q = model.addVars(len(aux), vtype = GRB.BINARY, name = "q")
#p = model.addVars(len(aux), vtype = GRB.BINARY, name = "p")
#r = model.addVars(len(aux), vtype = GRB.BINARY, name = "r")
model.update()

model.addConstrs((m[i,0] == 1) for i in R)
model.addConstrs((m[i,1] == 1) for i in F)
model.addConstrs((m[i,2] == 1) for i in G)

#Objective function is to maximize recovery bit and minimize the number of FIXED bit
model.setObjectiveN(gp.quicksum((m[i,0]) for i in range(state_len)),0,1, GRB.MAXIMIZE)
model.setObjectiveN(gp.quicksum((m[i,1]) for i in range(state_len)),1,0, GRB.MINIMIZE)

model.addConstrs((x[i] == (2*m[i,0] + 3*m[i,1] + 4*m[i,2])) for i in range(state_len))
model.addConstrs(m[i,0] + m[i,1] + m[i,2] == 1 for i in range(state_len))

#Number of recovered bit should be less than or equal to the number of keystream equations
#model.addConstr(gp.quicksum(m[i,0] for i in range(state_len)) == keys)
model.addConstr(gp.quicksum(m[i,0] for i in range(state_len) if i not in linear_all) == 0)

count = 0
cnt = 0
for loop in range(keys):
    z = Z[loop]
    #z_var contains the index of variables in Z
    #z_var = np.array(z.variables())
    #z_var = [g.index(i) for i in z_var]
    z_monom = np.array(z.monomials())

    z_linear = [g.index(i) for i in z_monom if i.degree() == 1]

    #z_nlinear = [list(i.variables()) for i in z_monom if len(i.variables()) > 1]
    #z_nlinear = [g.index(z_nlinear[i][j]) for i in range(len(z_nlinear)) for j in range(len(z_nlinear[i]))]
    #z_nlinear = list(set(z_nlinear))


    #linear_new = [i for i in z_linear if i not in (linear + z_nlinear)]
    #z_linear_org = [i for i in z_linear if i not in z_nlinear]


    #One of the linear term variables which is not in the previous linear term has one recovered bit
    model.addConstr(gp.quicksum(m[i,0] for i in linear[loop]) <= 1)    
    #model.addConstr(gp.quicksum(m[i,0] for i in z_linear) <= loop + 1) 



    for mon in z_monom:
        if mon.degree() > 1:
            var = list(mon.variables())
            var = [g.index(i) for i in var]
            inters = [0]*len(linear)
            n = [0]*len(linear)
            for l in range(len(linear)):
                inters[l] = intersection(var,linear[l])
                n[l] = len(inters[l])

            C_prev = [n[i] for i in range(loop)] 
            C_current = [n[loop]]
            C_future = [n[i] for i in range(loop+1,keys,1)]

            #Case 1: recovered var in mon and is in linear
            #if sum(C_prev) > 0 and sum(C_current) == 0 and sum(C_future) == 0 :
            #    #if recovery found then guess
            #    model.addConstr((p[count] == 0) >> (gp.quicksum(m[i,0] for i in var) == 0))
            #    model.addConstr((p[count] == 1) >> (gp.quicksum((m[i,1] + m[i,2]) for i in var) >= len(var) - sum(C_prev)))
            #    count += 1

            #Case 2: recovered var in mon and is in z_linear-linear
            if (sum(C_current + C_future) > 0):
                #if recovery found then fix to zero
                prev = []
                for k in range(loop):
                    prev = prev + inters[k]
                #var_ge = [i for i in var if i not in prev]
                
                var_ge = []
                for k in range(loop,len(linear),1):
                    var_ge = var_ge + inters[k]

                model.addConstr((q[count] == 0) >> (gp.quicksum(m[i,0] for i in var_ge) == 0))
                model.addConstr((q[count] == 1) >> (gp.quicksum(m[i,1] for i in var) >= 1))
                count += 1

            #if (sum(C_prev + C_current + C_future) == 0): 
            #    model.addConstr(gp.quicksum((m[i,0]) for i in var) == 0)
    print("2nd part loop ", loop)
print("2nd part done")
import time

model.params.outputflag = 0
start = time.time()
model.optimize()
end = time.time() - start

print("Keystream = ",keys)
print("status",model.STATUS)


f = open(filename,'a')
f.write("\nKeystream = " + str(keys))
f.write("\nStatus " + str(model.STATUS) + "\n")

    
if model.STATUS == 2:
    val = model.x
    R = []
    F = []
    G = []
    for i in range(state_len):
        if round(val[i]) == 2:
            R.append(i)
        if round(val[i]) == 3:
            F.append(i)
        if round(val[i]) == 4:
            G.append(i)



    Rec = [g[i] for i in R]
    Fix = [g[i] for i in F]
    Guess = [g[i] for i in G]
    print("Recovery Bit ",Rec)
    print("Fixed Bit ",Fix)
    print("Guessed Bit ",Guess)
    
    f.write(str("Recovery Bit ") + str(Rec) + "\n")
    f.write(str("Fixed Bit ") + str(Fix) + "\n")
    f.write(str("Guessed Bit ") + str(Guess) + "\n")
        
        
    print(len(R),len(F))
    if (len(R) + len(F) + len(G) == state_len) and len(intersection(R,F) + intersection(F,G) + intersection(R,G)) == 0:
        print("Length verified")
        f.write("\nLength Verified\n\n")
end2 = time.time()-start2
print("Gurobi Time = ", end)
print("Overall Time = ", end2)

f.write("\n\nGurobi Time = " + str(end))
f.write("\nOverall Time = " + str(end2))
f.close()

Defining s0, s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60, s61, s62, s63, s64, s65, s66, s67, s68, s69, s70, s71, s72, s73, s74, s75, s76, s77, s78, s79, s80, s81, s82, s83, s84, s85, s86, s87, s88, s89, s90, s91, s92, s93, s94, s95, s96, s97, s98, s99, s100, s101, s102, s103, s104, s105, s106, s107, s108, s109, s110, s111, s112, s113, s114, s115, s116, s117, s118, s119, s120, s121, s122, s123, s124, s125, s126, s127, s128, s129, s130, s131, s132, s133, s134, s135, s136, s137, s138, s139, s140, s141, s142, s143, s144, s145, s146, s147, s148, s149, s150, s151, s152, s153, s154, s155, s156, s157, s158, s159, s160, s161, s162, s163, s164, s165, s166, s167, s168, s169, s170, s171, s172, s173, s174, s175, s176, s177, s178, s179, s180, s181, s182, s18

In [3]:
R = [81, 85, 89, 101, 143, 144, 145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 187, 225, 226]
F = [29, 30, 31, 32, 33, 34, 35, 36, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
G = [i for i in range(state_len) if i not in R + F]
print(len(F))

38


In [ ]:
#keys = len(R)
S = g[:]

#########################Fixed bit##########################
for i in F:
    S[i] = 0


Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)



R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys):
    z = keystream(S)
    monom = list(z.monomials())
    t_var = list(z.variables())

    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            f = open(filename,'a')
            f.write("\nError --- Future recovery bits found\n" + str(m))
            f.close()
            print("Error --- Future recovery bits found")
            print(m)

    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    #print("Recovery bits till now in linear terms ",L_rec)

    
    S = state_update(S)

f = open(filename,'a')
if flag == 0:
    print("Failed")
    f.write("\nFailed\n")
else:
    print("Passed")
    f.write("\nPassed\n")

    import csv
    with open('Espresso_TMDTO_MILP_DATA.csv','a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([len(R),len(F),Rec,Fix])
f.close()

In [ ]:
#keys = len(R)
S = g[:]
R = [i for i in range(137,172,1)]
#G = [i for i in range(29)] + [i for i in range(37,46,1)] + [i for i in range(77,137,1)] + [i for i in range(172,256,1)]
F = [i for i in range(29,37,1)]
G = [i for i in range(state_len) if i not in R + F]
#########################Fixed bit##########################
for i in F:
    S[i] = 0


Rec = [g[i] for i in R]
print("Recovered Bits are ", Rec)
Fix = [g[i] for i in F]
print("Fixed Bits are ", Fix)



R_past = []
L_rec = []
cnt = 0
flag = 1
for i in range(keys):
    z = keystream(S)
    monom = list(z.monomials())
    t_var = list(z.variables())

    R_past = R_past + list(set(L_rec))
    m_rec = []
    for m in monom:
        m_var = list(m.variables())
        if (len(m_var) == 1) and (len(intersection(m_var,Rec)) > 0):
            L_rec = L_rec + m_var

        if (len(m_var) > 1) and (len(intersection(m_var,Rec)) > 0):
            m_rec = m_rec + [m]
    #print("monomials containing recovery bits (degree > 1) ",m_rec)

    for m in m_rec:
        m_var = list(m.variables())
        #Now L_rec contains current and past recovery bit in linear terms
        #Whereas R_past contains only the past recovery bit in linear terms
        if len(intersection(m_var,Rec)) > len(intersection(m_var,R_past)):
            f = open(filename,'a')
            f.write("\nError --- Future recovery bits found\n" + str(m))
            f.close()
            print("Error --- Future recovery bits found")
            print(m)

    L_rec = list(set(L_rec))
    if len(L_rec) != cnt + 1:
        flag = 0
    cnt = len(L_rec)
    print("Recovery bits till now in linear terms ",L_rec)

    
    S = state_update(S)

f = open(filename,'a')
if flag == 0:
    print("Failed")
    f.write("\nFailed\n")
else:
    print("Passed")
    f.write("\nPassed\n")

    import csv
    with open('Espresso_TMDTO_MILP_DATA.csv','a') as f:
        writer = csv.writer(f)
        #writer.writerow(['Number of Recovered bits','Number of Fixed bits','Recovered bits','Fixed bits'])
        writer.writerow([len(R),len(F),Rec,Fix])
f.close()

In [ ]:
R = [x[i] for i in range(137,172,1)]
temp = [x[i] for i in range(0,29,1)] + [x[i] for i in range(37,46,1)] \
+ [x[i] for i in range(77,137,1)] + [x[i] for i in range(172,256,1)]

F = [x[i] for i in range(29,37,1)] + [x[i] for i in range(46,77,1)]
G = temp[:]
if len(R+F+G) == state_size:
    print("matched")
    
    
S = x[:]
for i in F:
    S[x.index(i)] = 0
keys = len(R)
for i in range(keys):
    z = keystream(S)
    z_monom = list(z.monomials())
    
    print("Rounds ", i)
    for mon in z_monom:
        var = mon.variables()
        if intersection(var,R) >= 1:
            print(mon,end = " + ")
    print("\n")
    S = state_update(S)
        

In [ ]:
R = [i for i in range(137,172,1)]
G = [i for i in range(29)] + [i for i in range(37,46,1)] + [i for i in range(77,137,1)] + [i for i in range(172,256,1)]
F = [i for i in range(256) if i not in R + G]
print(F)
print(len(F))

In [ ]:
import threading
import multiprocessing
  
def print_cube(num):
    print("Cube:",num * num * num)

def print_square(num):
    print("Square: ",num * num)
    
import time
import timeit

def normal():
    print_square(10)
    print_cube(10)


def thread():
    t1 = threading.Thread(target=print_square, args=(10,))
    t2 = threading.Thread(target=print_cube, args=(10,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

def multiprocess():
    t1  = multiprocessing.Process(target=print_square, args=(10,))
    t2  = multiprocessing.Process(target=print_cube, args=(10,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(timeit.timeit(normal,number = 1))
print(timeit.timeit(thread,number = 1))
print(timeit.timeit(multiprocess,number = 1))

In [ ]:
import time
import threading
import multiprocessing

def fn():
    '''since all 3 functions were identical you can just use one ...'''
    x = 0
    while  x < 100000:
        x += 1
    



def TEST_THREADS():
    new_thread1  = threading.Thread(target = fn , args = ())
    new_thread2  = threading.Thread(target = fn, args = ())
    new_thread3  = threading.Thread(target = fn, args = ())
    new_thread1.start()
    new_thread2.start()
    new_thread3.start()
    new_thread1.join()
    new_thread2.join()
    new_thread3.join()

def TEST_NORMAL():
    fn()
    fn()
    fn()
    
def TEST_MULTIPROCESSING():
    new_thread1  = multiprocessing.Process(target = fn , args = ())
    new_thread2  = multiprocessing.Process(target = fn, args = ())
    new_thread3  = multiprocessing.Process(target = fn, args = ())
    new_thread1.start()
    new_thread2.start()
    new_thread3.start()
    new_thread1.join()
    new_thread2.join()
    new_thread3.join
if __name__ == "__main__":  
    '''It is very important to use name == __main__ guard code with threads and multiprocessing'''
    import timeit
    print("Time to Run 1x: %0.2fs"%(timeit.timeit(fn,number=1),))
    print("NORMAL:%0.2fs"%(timeit.timeit(TEST_NORMAL,number=1),))
    print("Threaded: %0.2fs"%(timeit.timeit(TEST_THREADS,number=1),))
    print("Multiprocessing: %0.2fs"%(timeit.timeit(TEST_MULTIPROCESSING,number=1),))
